In [1]:

import numpy as np
import scipy as sp
import h5py

import os

import SimData as sim_data
import Simulator as sim_system
import Optimization as opt

In [2]:


###* Input files and function

reactions_file = "reactionsSimpleV1.json"

const_dict = {
        "F0": 1.5e15,           # cm^-2
        "S0": 3e13,             # cm^-2
        
        "R": 0.00831442,        # kJ/mol*K
        "kBoltz": 1.380649e-23, # J/K
}

initial_state_dict = {'O_F': 0.1, 'O2_F':0.1 ,'O_S': 0.1, 'Vdb_S':0.1, 
                      'Odb_S': 0.1, 'CO_F': 0.1, 'CO2_F':0.1, 'CO_S': 0.1, 
                      'COdb_S': 0.0}

###* Functions for the data transformation
def compute_flux(const_dict, exp_dict, specie, molar_mass):
    den = exp_dict.get(specie, 0.0)
    v_th = np.sqrt((8.0 * const_dict['R'] * 1000 * exp_dict['Tnw'])/(molar_mass * np.pi))
    flux = 0.25 * v_th * den * 100
    return flux


def compute_remaining_flux(const_dict, exp_dict, molar_mass): 
    den = exp_dict['N'] - exp_dict['O'] - exp_dict['CO']
    v_th = np.sqrt((8.0 * const_dict['R'] * 1000 * exp_dict['Tnw'])/(molar_mass * np.pi))
    flux = 0.25 * v_th * den * 100
    return flux

####? EavgMB data extracted from the Booth et al. 2019 paper
p_data_exp = [0.2, 0.3, 0.4, 0.5, 0.6, 0.75, 1.5]
EavgMB_data = [1.04, 0.91, 0.87, 0.83, 0.77, 0.5, 0.001]
interpolator = sp.interpolate.interp1d(p_data_exp, EavgMB_data, kind='linear', fill_value=0.001, bounds_error=False)


transformations_exp = {
    'Tw':       lambda const_dict, exp_dict: exp_dict['Tw'] + 273.15,
    'fluxO' :   lambda const_dict, exp_dict: compute_flux(const_dict, exp_dict,'O', 0.016),
    'fluxO2' :  lambda const_dict, exp_dict: compute_flux(const_dict, exp_dict,'O2', 0.032),
    'fluxO3' :  lambda const_dict, exp_dict: compute_flux(const_dict, exp_dict,'O3', 0.048),
    'fluxC':    lambda const_dict, exp_dict: compute_flux(const_dict, exp_dict, 'C', 0.012),
    'fluxCO':   lambda const_dict, exp_dict: compute_flux(const_dict, exp_dict, 'CO', 0.028),
    'fluxCO2':  lambda const_dict, exp_dict: compute_flux(const_dict, exp_dict, 'CO2', 0.048),
    'EavgMB':   lambda const_dict, exp_dict: interpolator(exp_dict['pressure']).item(),
    'Ion':      lambda const_dict, exp_dict: 1e14 * exp_dict["current"]
}

output_folder_path = "Buffer_Data"
exp_data_file = "Experimental_data_CO_Jorge.hdf5"

In [3]:


def func_optimization(params):
    
    A_O, B_O, E_O, A_CO, B_CO, E_CO, SF_1, SF_2 = params
    
    nu_d_mod = lambda T: 1e15 * (A_O + B_O * np.exp(E_O/(const_dict['R'] * T)))
    nu_d_mod_2 = lambda T: 1e15 * (A_CO + B_CO * np.exp(E_CO/(const_dict['R'] * T)))
    
    dict_mod_vec = [
    {"id": 2, "rate": None, "model_dict": {"nu_d": nu_d_mod}},
    {"id": 10, "rate": None, "model_dict": {"nu_d": nu_d_mod}},
    {"id": 31, "rate": None, "model_dict": {"nu_d": nu_d_mod_2}},
    
    {"id": 32, "rate": None, "model_dict": {"SF": SF_1}},
    {"id": 35, "rate": None, "model_dict": {"SF": SF_1}},
    
    {"id": 33, "rate": None, "model_dict": {"SF": SF_2}},
    {"id": 34, "rate": None, "model_dict": {"SF": SF_2}},
    {"id": 36, "rate": None, "model_dict": {"SF": SF_2}},
    {"id": 37, "rate": None, "model_dict": {"SF": SF_2}},
    {"id": 38, "rate": None, "model_dict": {"SF": SF_2}},
    {"id": 39, "rate": None, "model_dict": {"SF": SF_2}},
    ]
    
    return dict_mod_vec

loss_function = lambda exp, teo: np.mean((np.reciprocal(exp)*(exp-teo))**2)

exp_file = os.path.join(output_folder_path, exp_data_file)
sim = sim_system.Simulator(reactions_file, const_dict, exp_file, initial_state_dict, transformations_exp=transformations_exp)


  d[CO2_F]/dt = -CO2_F*r_29 + r_28*(-CO2_F - CO_F - O2_F - O_F + 1.0)
  d[CO_F]/dt = -CO_F*O_F*r_34 - 0.02*CO_F*O_S*r_39 - CO_F*r_31 - CO_F*r_33 - 0.02*CO_F*r_35*(-CO_S - O_S - Odb_S - Vdb_S + 1.0) + r_30*(-CO2_F - CO_F - O2_F - O_F + 1.0)
  d[CO_S]/dt = CO_F*r_35*(-CO_S - O_S - Odb_S - Vdb_S + 1.0) - CO_S*O_F*r_38 - CO_S*r_36 + r_32*(-CO_S - O_S - Odb_S - Vdb_S + 1.0)
  d[O2_F]/dt = -O2_F*O_F*r_15 - O2_F*r_10 - O2_F*r_12 - O2_F*r_14 + r_9*(-CO2_F - CO_F - O2_F - O_F + 1.0)
  d[O_F]/dt = -CO_F*O_F*r_34 - 0.02*CO_S*O_F*r_38 - O2_F*O_F*r_15 - 2*O_F**2*r_8 - 0.02*O_F*O_S*r_7 - 0.02*O_F*Odb_S*r_27 - 0.02*O_F*Vdb_S*r_26 - O_F*r_11 - O_F*r_2 - O_F*r_4 - 0.02*O_F*r_5*(-CO_S - O_S - Odb_S - Vdb_S + 1.0) + r_1*(-CO2_F - CO_F - O2_F - O_F + 1.0)
  d[O_S]/dt = -CO_F*O_S*r_39 - O_F*O_S*r_7 + O_F*r_5*(-CO_S - O_S - Odb_S - Vdb_S + 1.0) - O_S*r_16 - O_S*r_17 - O_S*r_37 - O_S*r_6 + r_3*(-CO_S - O_S - Odb_S - Vdb_S + 1.0)
  d[Odb_S]/dt = -O_F*Odb_S*r_27 + O_F*Vdb_S*r_26 - Odb_S*r_23 - Odb_S*r_24 - Odb

In [4]:

if __name__ == '__main__':
    optimizer = opt.Optimizer(sim, func_optimization, loss_function)

    global_bounds = [(1e-5, 1.0), (1e-8, 1.0), (5, 30), (1e-5, 1.0), (1e-8, 1.0), (5, 30), \
                    (4e-4, 0.2), (1e-2, 1.0)]

    config = {
        "bounds": global_bounds,
        "de_maxiter": 3,
        "de_num_iterations": 1,
    }
    
    best_local, best_local_loss = optimizer.hybrid_optimization_searchV2(config)

# differential_evolution step 50: f(x)= 0.041662857183863554


differential_evolution step 1: f(x)= 0.21071210453236106
differential_evolution step 2: f(x)= 0.1537843213116148
differential_evolution step 3: f(x)= 0.1537843213116148
differential_evolution step 4: f(x)= 0.1537843213116148
differential_evolution step 5: f(x)= 0.1537843213116148
differential_evolution step 6: f(x)= 0.1537843213116148
differential_evolution step 7: f(x)= 0.10299461195749077
differential_evolution step 8: f(x)= 0.10299461195749077
differential_evolution step 9: f(x)= 0.10299461195749077
differential_evolution step 10: f(x)= 0.10299461195749077
differential_evolution step 11: f(x)= 0.09595094650859375
differential_evolution step 12: f(x)= 0.09595094650859375
differential_evolution step 13: f(x)= 0.09595094650859375


differential_evolution step 14: f(x)= 0.09595094650859375
differential_evolution step 15: f(x)= 0.08807196601881409
differential_evolution step 16: f(x)= 0.08807196601881409
differential_evolution step 17: f(x)= 0.08807196601881409
differential_evolution step 18: f(x)= 0.0809414670156313
differential_evolution step 19: f(x)= 0.0809414670156313
differential_evolution step 20: f(x)= 0.0809414670156313
differential_evolution step 21: f(x)= 0.07220329653755296
differential_evolution step 22: f(x)= 0.06315364853871847
differential_evolution step 23: f(x)= 0.06315364853871847
differential_evolution step 24: f(x)= 0.06315364853871847
differential_evolution step 25: f(x)= 0.06315364853871847
differential_evolution step 26: f(x)= 0.06315364853871847
differential_evolution step 27: f(x)= 0.06315364853871847
differential_evolution step 28: f(x)= 0.06315364853871847


differential_evolution step 29: f(x)= 0.06315364853871847


differential_evolution step 30: f(x)= 0.06315364853871847
differential_evolution step 31: f(x)= 0.06315364853871847


differential_evolution step 32: f(x)= 0.06315364853871847
differential_evolution step 33: f(x)= 0.06315364853871847
differential_evolution step 34: f(x)= 0.06315364853871847
differential_evolution step 35: f(x)= 0.06315364853871847


...

differential_evolution step 56: f(x)= 0.049102554533180166
differential_evolution step 57: f(x)= 0.049102554533180166
differential_evolution step 58: f(x)= 0.047756311379645484


differential_evolution step 59: f(x)= 0.047756311379645484
differential_evolution step 60: f(x)= 0.04712446224218804
differential_evolution step 61: f(x)= 0.04671110984993845
differential_evolution step 62: f(x)= 0.0462733565975873
differential_evolution step 63: f(x)= 0.04557630285534677


differential_evolution step 64: f(x)= 0.04557630285534677


differential_evolution step 65: f(x)= 0.04432619685608442
differential_evolution step 66: f(x)= 0.0435841084787234
differential_evolution step 67: f(x)= 0.043445237240822755
differential_evolution step 68: f(x)= 0.04265391769027569
differential_evolution step 69: f(x)= 0.04265391769027569
differential_evolution step 70: f(x)= 0.042545949431174304
differential_evolution step 71: f(x)= 0.04206189184200132
differential_evolution step 72: f(x)= 0.04178315981574201
differential_evolution step 73: f(x)= 0.04178315981574201
differential_evolution step 74: f(x)= 0.041592220413429276
differential_evolution step 75: f(x)= 0.041458107411516065
differential_evolution step 76: f(x)= 0.04126779148889978
differential_evolution step 77: f(x)= 0.041136346433818
differential_evolution step 78: f(x)= 0.041136346433818
differential_evolution step 79: f(x)= 0.041136346433818


differential_evolution step 80: f(x)= 0.04104801125934555
top_k_mod:  12
res:  0.04125195850550937 [5.05400495e-04 1.25720940e-03 1.51916654e+01 5.93548432e-01
 7.13883866e-01 1.18344768e+01 4.15599417e-02 6.67854869e-01] [5.05400495e-04 1.25720940e-03 1.51916654e+01 5.93548432e-01
 7.13883866e-01 1.18344768e+01 4.15599417e-02 6.36052256e-01]
res:  0.04102696093862821 [4.09903828e-03 1.18390562e-03 1.53637837e+01 3.43796400e-01
 6.11472254e-01 2.23511371e+01 4.17940678e-02 8.12984010e-01] [4.09903828e-03 1.18390562e-03 1.53637837e+01 3.43796400e-01
 6.11472254e-01 2.23511371e+01 3.98038741e-02 8.12984010e-01]
res:  0.041297504121159616 [6.34723968e-03 1.27109273e-03 1.52078467e+01 6.10433466e-01
 6.27993326e-01 1.28502315e+01 3.25551501e-02 7.45268500e-01] [6.34723968e-03 1.27109273e-03 1.52078467e+01 6.10433466e-01
 6.27993326e-01 1.28502315e+01 3.10049048e-02 7.45268500e-01]
res: 
 0.04105266025342026 [4.87706965e-03 1.15656072e-03 1.53942209e+01 4.50921062e-01
 8.79103407e-01 2.6543

In [5]:
print(best_local)
print(best_local_loss)

             message: Maximum number of iterations has been exceeded.
             success: False
                 fun: 0.04104801125934555
                   x: [ 4.099e-03  1.184e-03  1.536e+01  3.438e-01
                        6.115e-01  2.235e+01  3.980e-02  8.130e-01]
                 nit: 80
                nfev: 9720
          population: [[ 4.099e-03  1.184e-03 ...  3.980e-02  8.130e-01]
                       [ 1.341e-02  1.622e-03 ...  1.159e-02  3.105e-01]
                       ...
                       [ 1.924e-02  1.732e-03 ...  3.338e-02  8.113e-01]
                       [ 4.834e-03  9.254e-04 ...  5.363e-02  6.425e-01]]
 population_energies: [ 4.105e-02  4.563e-02 ...  4.648e-02  4.502e-02]
             message: Optimization terminated successfully.
             success: True
                 fun: 0.041019383761423125
                   x: [ 1.612e-03  1.188e-03  1.536e+01  3.438e-01
                        6.115e-01  2.235e+01  4.213e-02  8.130e-01]
                

In [6]:

# def func_optimization(params):
    
#     A_O, B_O, E_O, A_CO, B_CO, E_CO, SF_1, SF_2 = params
    
#     nu_d_mod = lambda T: 1e15 * (A_O + B_O * np.exp(E_O/(const_dict['R'] * T)))
#     nu_d_mod_2 = lambda T: 1e15 * (A_CO + B_CO * np.exp(E_CO/(const_dict['R'] * T)))
    
#     dict_mod_vec = [
#     {"id": 2, "rate": None, "model_dict": {"nu_d": nu_d_mod}},
#     {"id": 10, "rate": None, "model_dict": {"nu_d": nu_d_mod}},
#     {"id": 31, "rate": None, "model_dict": {"nu_d": nu_d_mod_2}},
    
#     {"id": 32, "rate": None, "model_dict": {"SF": SF_1}},
#     {"id": 35, "rate": None, "model_dict": {"SF": SF_1}},
    
#     {"id": 33, "rate": None, "model_dict": {"SF": SF_2}},
#     {"id": 34, "rate": None, "model_dict": {"SF": SF_2}},
#     {"id": 36, "rate": None, "model_dict": {"SF": SF_2}},
#     {"id": 37, "rate": None, "model_dict": {"SF": SF_2}},
#     {"id": 38, "rate": None, "model_dict": {"SF": SF_2}},
#     {"id": 39, "rate": None, "model_dict": {"SF": SF_2}},
#     ]
    
#     return dict_mod_vec ----->

#          message: Maximum number of iterations has been exceeded.
#              success: False
#                  fun: 0.04104801125934555
#                    x: [ 4.099e-03  1.184e-03  1.536e+01  3.438e-01
#                         6.115e-01  2.235e+01  3.980e-02  8.130e-01]
#                  nit: 80
#                 nfev: 9720
#           population: [[ 4.099e-03  1.184e-03 ...  3.980e-02  8.130e-01]
#                        [ 1.341e-02  1.622e-03 ...  1.159e-02  3.105e-01]
#                        ...
#                        [ 1.924e-02  1.732e-03 ...  3.338e-02  8.113e-01]
#                        [ 4.834e-03  9.254e-04 ...  5.363e-02  6.425e-01]]
#  population_energies: [ 4.105e-02  4.563e-02 ...  4.648e-02  4.502e-02]
#              message: Optimization terminated successfully.
#              success: True
#                  fun: 0.041019383761423125
#                    x: [ 1.612e-03  1.188e-03  1.536e+01  3.438e-01
#                         6.115e-01  2.235e+01  4.213e-02  8.130e-01]
#                  nit: 1
#                 nfev: 195
#           population: [[ 4.099e-03  1.184e-03 ...  4.179e-02  8.130e-01]
#                        [ 4.877e-03  1.157e-03 ...  5.353e-02  8.593e-01]
#                        ...
#                        [ 1.682e-02  1.231e-03 ...  3.234e-02  6.765e-01]
#                        [ 1.030e-02  1.243e-03 ...  3.989e-02  8.269e-01]]
#  population_energies: [ 4.102e-02  4.105e-02  4.123e-02  4.125e-02
#                         4.115e-02  4.138e-02  4.105e-02  4.141e-02
#                         4.149e-02  4.148e-02  4.149e-02  4.126e-02]
#                  jac: [-7.380e-04  6.570e-03 -5.492e-04  7.702e-08
#                         4.025e-08  6.870e-08 -6.396e-03  2.414e-04]





#   fun: 0.04481267533902918
#                    x: [ 1.000e-05  1.788e-03  1.427e+01  3.978e-01
#                         2.591e-01  2.163e+01]


# DE resul: [7.54561369e-02 3.30911944e-04 1.80716671e+01 6.55878140e-02
#  9.49664603e-01 5.15676107e+00] | 0.04027624574740497



#              message: Maximum number of iterations has been exceeded.
#              success: False
#                  fun: 0.04132735187758759
#                    x: [ 2.584e-02  6.806e-04  1.656e+01  6.329e-01
#                         8.919e-01  1.696e+01]
#                  nit: 50
#                 nfev: 4590
#           population: [[ 2.584e-02  6.806e-04 ...  8.919e-01  1.696e+01]
#                        [ 2.777e-02  1.851e-03 ...  6.498e-01  1.415e+01]
#                        ...
#                        [ 1.123e-01  1.465e-03 ...  2.758e-01  2.268e+01]
#                        [ 9.516e-02  1.558e-03 ...  7.147e-01  1.032e+01]]
#  population_energies: [ 4.133e-02  5.156e-02 ...  6.500e-02  6.471e-02]
#              message: Optimization terminated successfully.
#              success: True
#                  fun: 0.0412456072148422
#                    x: [ 2.649e-02  6.797e-04  1.653e+01  9.830e-01
#                         8.385e-02  2.636e+01]
#                  nit: 1
#                 nfev: 18
#           population: [[ 2.649e-02  6.797e-04 ...  8.385e-02  2.636e+01]
#                        [ 3.264e-02  6.970e-04 ...  8.901e-02  2.424e+01]
#                        ...
#                        [ 9.326e-03  7.979e-04 ...  1.930e-01  2.220e+01]
#                        [ 6.376e-02  6.047e-04 ...  1.204e-01  2.659e+01]]
#  population_energies: [ 4.125e-02  4.135e-02  4.137e-02  4.161e-02
#                         4.146e-02  4.133e-02  4.187e-02  4.155e-02
#                         4.231e-02]

# ####* Test default 

import matplotlib.pyplot as plt


def do_plots(params):


    loss = optimizer.objective_function(params)
    print("PARAMS: ", params, "Loss: ", loss)

    frac_solutions_arr, gammas_results_arr, gammas_sum_arr, gamma_exp_data = optimizer.solve_simulations_updated(params)
    exp_data_arr = optimizer.exp_data_arr

    inv_Tnw_vec = np.array([300/vec['Tnw'] for vec in exp_data_arr])

    plt.figure
    plt.plot(inv_Tnw_vec, gamma_exp_data, '.', label="Exp")
    plt.plot(inv_Tnw_vec, gammas_sum_arr, '.', label="Sim")
    plt.legend()
    plt.xlabel(r"$300/T_{nw}$")
    plt.ylabel(r"$\gamma$")

    current_vec = np.array([vec['current'] for vec in exp_data_arr])
    Tw_vec = np.array([vec['Tw'] for vec in exp_data_arr])

    current_unique = np.unique(current_vec)
    Tw_unique = np.unique([vec['Tw'] for vec in exp_data_arr])


    A, B = np.meshgrid(current_unique, Tw_unique)
    combinations = np.stack([A.ravel(), B.ravel()], axis=1)

    for comb in combinations:
        current = comb[0]
        Tw = comb[1]

        idx_valid = np.logical_and(current_vec == current, Tw_vec == Tw)
        data_arr = exp_data_arr[idx_valid]
        
        pressure_vec = np.array([vec['pressure'] for vec in data_arr])
        gamma_exp = gamma_exp_data[idx_valid]
        gamma_sim = gammas_sum_arr[idx_valid]
        
        plt.figure()
        plt.plot(pressure_vec, gamma_exp*1e4, 's', label="Exp")
        plt.plot(pressure_vec, gamma_sim*1e4, 's', label="Sim")
        plt.title(f"I = {current} and Tw = {Tw}")
        plt.xlabel("Pressure")
        plt.ylabel(r"$\gamma$")
        plt.legend()
        plt.grid()
        plt.ylim(1, 14)
        plt.show()



In [7]:

# params_1 = np.array([0.01634, 1.67e-4, 19.75])
# do_plots(params_1)


In [8]:





# params_2 = np.array([2.39375716e-02, 7.11846527e-04, 1.64416626e+01])
# do_plots(params_2)

In [9]:

# def nu_d_func(T, params):
#     A, B, E = params
    
#     nu_d_mod = 1e15 * (A + B * np.exp(E * np.reciprocal(const_dict['R'] * T)))
    
#     return nu_d_mod


# T_space = np.linspace(-20+273.15, 50+273.15, 200)

# vec_1 = np.array([nu_d_func(T, params_1) for T in T_space])
# vec_2 = np.array([nu_d_func(T, params_2) for T in T_space])
# vec_3 = np.array([nu_d_func(T, params_3) for T in T_space])

# plt.plot(T_space, vec_1, label="Paper")
# plt.plot(T_space, vec_2, label="DE 1")
# plt.plot(T_space, vec_3, label="DE 2")
# plt.legend()
# plt.xlabel("Tw")
# plt.ylabel(r"$\nu_{d}$")